# Introducción

Un problema de **optimización** es uno en el que, además de proporcionar unos requisitos para que una solución sea factible, se proporciona un criterio para _cuantificar lo buena o lo mala que es cada solución factible hasta obtener una solución_

### Los **algoritmos voraces** suelen funcionar muy bien para resolver algunos problemas de optimización...
Pero requieren del cumplimiento de ciertas condiciones:
- Como son algoritmos **miopes**, que no ven a largo plazo, necesitan tomar la mejor decisión local y que..
- Esta elección conduzca eventualmente a una optimización global del problema.

Podemos clasificar los **tipos de algoritmos voraces**
- Si no consiguen una solución óptima pero se acercan, decimos que son algoritmos de aproximación.
- Si la consiguen diremos que el algoritmo es **heurístico** (porque la heurística que siguen es la correcta hacia una resolución óptima del problema).

# Problemas

## _1. Mochila con fraccionamiento_


Una variante del problema de la mochila en la que tenemos una "espada mágica" con la que partir los objetos para llevarnos sólo una fracción de los mismos. Esto nos permite llenar la mochila por completo **siempre**.

Definamos como es costumbre el espacio de soluciones:

$ X = \left\{ x_1, x_2, x_3, ..., x_N \right\} \in \left[0,1\right]^\mathbb{N} | \sum\limits_{1 \leq i \leq N}{x_iw_i} \leq C$

Es decir, serán soluciones al problema aquellos arreglos de N elementos de 0s y 1s (referenciando ésto a los objetos seleccionados) cuya suma de pesos sea menor a la capacidad total de la mochila.

Se intenta maximizar la función siguiente:

$ f(x_1, x_2, ... x_N) = \sum\limits_{1 \leq i \leq N}{x_iv_i}$

Donde $ v_i $ es el valor de cada objeto del array.

Para maximizarla...

$ \hat{x} = \argmax\limits_{x_1, x_2, ... x_N} = f(x_1, x_2, ... x_N)$

La solución óptima de la mochila con fraccionamiento tiene un coste que viene dominado por la ordenación de los objetos por su beneficio unitario (**relación valor/peso**). Por tanto, para una entrada con N objetos, el coste del algoritmo es $ O(N \log N) $.

Aquí presentamos el código:

In [22]:
def coste_unit(wv):
  w,v= wv
  return v/w

def mochila_frac_optima(w,v,W):
  B = 0
  for wi, vi in sorted(zip(w, v), key=coste_unit, reverse=True):
    # Esto nos asegura tomar siempre o bien el objeto entero (1),
    # o parte del objeto (W/wi), pero nunca más de 1.
    ri = min(1, W/wi)
    # print(W/wi). Comprobar el comentario previo descomentando este print.
    W -= wi*ri
    B += ri*vi
  return B

Cuando usas simplemente `sorted(zip(w, v), reverse=True)`, **la ordenación se basa en las tuplas `(wi, vi)` directamente**. Específicamente:

- Primero, las tuplas se comparan por su primer elemento (`wi`), que corresponde a los valores de la lista `w`.
- Si hay empates en el primer elemento, se comparan por el segundo (`vi`).

En este caso, la ordenación prioriza los valores en `w`, no en la relación entre `w` y `v`.

Al usar `key=coste_unit`, le indicas a `sorted` que, en lugar de usar las tuplas directamente, debe calcular un valor derivado usando la función `coste_unit`. Supongamos que `coste_unit` está definida como:

```python
def coste_unit(pair):
    wi, vi = pair
    return vi / wi

In [23]:
w = [3, 4, 5, 9, 10]
v = [10, 40, 30, 50, 60]
W = 25

mochila_frac_optima(w,v,W)

163.33333333333331

## _2.Selección de actividades_ 

Imaginemos que queremos obtener el mayor beneficio posible alquilando una sala para realizar actividades.
- Cada actividad ocupa un intervalo de tiempo con hora de inicio $ s $ y una hora de terminación $ t $.
- Sólo puede llevarse a cabo una actividad en cada instante, pero podemos empezar una actividad justo cuando otra termina.
- El beneficio que nos reporta cada actividad es **el mismo independientemente de su duración**.

Podemos expresar estas actividades de la manera siguiente...

$ C = \left\{ (s_1, t_1), (s_2, t_2), ..., (s_N, t_N)\right\} $

El conjunto de soluciones factibles X está formado por los subconjuntos de C tales que si $ (s_i, t_i),(s_j, t_j) $ son elementos del conjunto, entonces **no se solapan**. Es decir, $ s_i \le t_i \leq s_j \le t_j $ 

Queremos **maximizar** el número de actividades que se lleven a cabo.

Ahora mostramos el código que resuelve el problema:

In [25]:
def coste_unit(st):
  s,t= st
  return t-s

def mochila_frac_optima(C):
  MAX = max(C, key=lambda x: x[1])[1]; 
  list = []; left = []; right = [];
  for si, ti in sorted(C, key=coste_unit, reverse=True):
    for l, r in zip(left, right):
      if (l < si and ti < r): continue
      list.append((si, ti))
      left.append(si); right.append(ti)
      break
  return 

  # Como esta solución es bastante mala, aquí dejamos otra

En esta solución se opta por ordenar el set en función del **orden de finalización de menor a mayor**. Nos fijamos en la condicón t_prev <= s que nos asegura seleccionar tareas cuyo tiempo de inicio sea MAYOR al anterior (siempre registrado por cada tarea que añadimos a la lista de solución). Si no son mayores, pasamos de largo.

El valor de inicialización de t_prev debe ser el menor de la lista SIEMPRE, ya que lógicamente todos los valores terminales de cada tarea serán mayores que este y **ordenar la lista por orden de terminación de menor a mayor nos asegura encontrar la tarea más corta con respecto al inicio de la planificación**.

A partir de este punto las iteraciones posteriores pueden verse como subestancias del problema, que intentan resolver lo mismo en un espacio de soluciones menor.

In [26]:
def seleccion_actividades(C):
  x, t_prev = set(), min(s for(s,t) in C)
  for s, t in sorted(C, key=lambda x: x[1]):
    if t_prev <= s:
      x.add((s,t))
      t_prev = t
  return x

<center>
<img src="./resources/voraz_actividades.png" alt="image">
</center>


## _3. El repostaje_

Un camión puede hacer $ n $ kms  con el depósito lleno.

Queremos hacer un viaje y conocemos la distancia entre cada gasolinera y la siguiente, ya que sabemos que todas están a menos de $ n $ kms entre sí.

El objetivo es parar el **mínimo número de veces a repostar**.

In [34]:
def gas_stations(M, d, n):
  stop = [0]
  km = n
  for i in range(M):
    if d[i] >= km:
      stop.append(i)
      km = n
    km -= d[i]
  stop.append(M+1)
  return stop

In [35]:
gas_stations(6, [65, 23, 45, 62, 12, 56, 26], 150)

[0, 3, 7]

Como es habitual con la estrategia voraz, hemos diseñado un algoritmo muy rápido: lineal con el número de gasolineras.